In [24]:
import requests # need to install requests package
from dotenv import load_dotenv # need to install python-dotenv package
import os
import json

import logging
logging.basicConfig(level=logging.INFO)

from pathlib import Path

data_dir = Path("./data/realtime")
data_dir.mkdir(parents=True, exist_ok=True)


In [3]:
# Never hard-code sensitive information like API tokens in your code.
# Instead, use environment variables to keep them secure.
# Make sure to create a .env file in the same directory as this script with the following content:
# ONC_API_TOKEN=your_api_token_here

load_dotenv()  # Load environment variables from .env file
onc_api_token = os.getenv("ONC_API_TOKEN")  # Load the ONC API token from the .env file

In [26]:
def save_json_data(json_data, device_code, data_dir, logging):
    """
    Saves the provided JSON data to a file in the specified directory.

    The filename is generated using the device code, measurement start time, and channel number.
    The JSON data is written with indentation for readability.

    Args:
        json_data (dict): The JSON data to save. Must contain keys for 'Resp' -> 'processed data' -> 'forward channel' and 'measurement start time'.
        device_code (str): The device code to include in the filename.
        data_dir (Path): The directory where the file will be saved.
        logging (logging.Logger): Logger for error and info messages.

    Raises:
        KeyError: If required keys are missing in json_data.
        TypeError: If json_data is not a dictionary.
        AttributeError: If start_time is not a string.
        Exception: If file writing fails.
    """
    try:
        chan = json_data["Resp"]["processed data"]["forward channel"] + 1
        start_time = json_data["Resp"]["processed data"]["measurement start time"]
        start_time = start_time.replace("-", "").replace(":", "")[:-5] + 'Z'
        filename = f"{device_code}_{start_time}_chan{chan}.json"
    except (KeyError, TypeError, AttributeError) as e:
        logging.error(f"Error extracting fields from json_data: {e}")
        raise

    try:
        with open(data_dir / filename, 'w') as f:
            json.dump(json_data, f, indent=4)
        logging.info(f"Generated filename: {filename}")
    except Exception as e:
        logging.error(f"Failed to write file {filename}: {e}")

In [ ]:

def process_and_save_json_lines(data, device_code, data_dir, logging):
    """
    Processes raw data lines from the ONC API response, extracts JSON data, and saves it to disk.

    Only lines starting with '{"Cmd":"getData",' are processed. Each valid JSON line is saved using save_json_data.

    Args:
        data (dict): The API response containing raw data lines under data['data'].
        device_code (str): Device code for filename generation.
        data_dir (Path): Directory to save JSON files.
        logging (logging.Logger): Logger for error/info messages.

    Returns:
        int: Number of JSON files successfully saved.

    Raises:
        Exception: If required keys are missing or JSON decoding fails.
    """
    saved_count = 0
    try:
        for raw_line in data.get('data', []):
            raw_data = raw_line.get('rawData', '')
            if raw_data.startswith('{"Cmd":"getData",'):
                try:
                    json_data = json.loads(raw_data)
                    save_json_data(json_data, device_code, data_dir, logging)
                    saved_count += 1
                except json.JSONDecodeError as e:
                    logging.error(f"JSON decode error: {e} in line: {raw_data}")
                except Exception as e:
                    logging.error(f"Error saving JSON data: {e}")
    except Exception as e:
        logging.error(f"Error processing raw lines: {e}")
        raise
    return saved_count


INFO:root:Generated filename: SILIXADTSXT19083_20250730T203539.535Z_chan2.json
INFO:root:Generated filename: SILIXADTSXT19083_20250730T203528.873Z_chan1.json


In [32]:
def fetch_onc_realtime_data(device_code, date_from, row_limit, size_limit, output_format, get_latest, onc_api_token, data_dir, logging):
    """
    Fetches ONC Realtime Data - raw readings for a given device and saves the response as JSON.

    Args:
        device_code (str): The device code to query.
        date_from (str): Start date for data retrieval (YYYY-MM-DD).
        row_limit (int): Maximum number of rows to retrieve.
        size_limit (int): Maximum size of data to retrieve.
        output_format (str): Format of the output data.
        get_latest (str): Whether to get the latest data ("true"/"false").
        onc_api_token (str): ONC API token for authentication.
        data_dir (Path): Directory to save data.
        logging (logging.Logger): Logger for info/error messages.

    Returns:
        dict: The JSON response from the API if successful, None otherwise.
    """
    url = "https://data.oceannetworks.ca/api/rawdata/device"
    params = {
        "deviceCode": device_code,
        "dateFrom": date_from,
        "rowLimit": row_limit,
        "sizeLimit": size_limit,
        "outputFormat": output_format,
        "getLatest": get_latest,
        "token": onc_api_token
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        # Optionally save the response to disk
        try:
            with open(data_dir / f"{device_code}_rawdata.json", "w") as f:
                json.dump(data, f, indent=4)
            logging.info(f"Saved raw data to {device_code}_rawdata.json")
        except Exception as e:
            logging.error(f"Failed to save raw data: {e}")
        return data
    else:
        logging.error(f"Error: {response.status_code} - {response.text}")
        return None


In [28]:
data['data'][1]['rawData']

'{"Cmd":"getData","Err":[],"Resp":{"path":"C:\\\\ProgramData\\\\Silixa\\\\XT DTS Data Acquisition\\\\temperature\\\\0000124000-0000124999\\\\0000124439.xt","software version":"XT DTS: 8.2.9.0 (64bit)","processed data":{"forward channel":1,"reverse channel":-1,"measurement start time":"2025-07-30T20:35:39.535+00:00","acquisition time":10.156000000000000583,"measurement complete":true,"probe temperatures":{"internal probe temperature":291.16294479960924946,"probe1 temperature":1112.3889999999998963,"probe2 temperature":1112.3709999999998672,"internal probe temperature 2":0},"probe voltage":{"internal probe voltage":0.10799286961555480679,"probe1 voltage":0.49999994039535522461,"probe2 voltage":0.49999994039535522461,"internal probe voltage 2":0},"forward signal data":{"Nav":0,"signal":{"Type":"","Dim":[],"Data":""}},"reverse signal data":{"Nav":0,"signal":{"Type":"","Dim":[],"Data":""}},"resampled forward raw data":{"z0":-224.02397211835233293,"dz":0.25419927587844254324,"signal":{"Type"

In [ ]:
# Look here for more information: https://data.oceannetworks.ca/OpenAPI#get-/rawdata/device

device_code = "SILIXADTSXT19083"
url = "https://data.oceannetworks.ca/api/rawdata/device"
params = {
    "deviceCode": device_code,  # Replace with the actual device code
    "dateFrom": "2025-07-28",
    "rowLimit": 20,
    "sizeLimit": 20,
    "outputFormat": "Object",
    "getLatest": "false",
    "token": onc_api_token
}

response = requests.get(url, params=params)
if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code} - {response.text}")